In [13]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from scipy.stats import chi2_contingency
from itertools import permutations, combinations, product``666666666666666666`
import math, random, time

random.seed(42)
np.random.seed(42)

### Load Dataset
We load the stroke dataset and view basic details.  
This gives us an idea of what data we will work with.


In [ ]:
df = pd.read_csv("E:\\Semester 5\\Probabilistic Reasoning\\Project\\archive\\healthcare-dataset-stroke-data.csv")
print("Rows,Cols:", df.shape)
df.head()

### Clean Data
We handle missing values and fix any inconsistent entries.  
This prepares the data for modeling.


In [15]:
cols_needed = ['age','hypertension','heart_disease','avg_glucose_level','bmi','smoking_status','stroke']
# if dataset has more features you may include them; keep this minimal for speed
for c in cols_needed:
    if c not in df.columns:
        raise ValueError(f"Column {c} not found, available columns: {df.columns.tolist()}")
df = df[cols_needed].copy()

# Impute BMI median
df['bmi'] = df['bmi'].fillna(df['bmi'].median())

# Simplify smoking_status (merge 'Unknown' with 'never smoked' if exists)
if 'smoking_status' in df.columns:
    df['smoking_status'] = df['smoking_status'].replace('Unknown', 'never smoked')

# Convert stroke to string labels
df['stroke'] = df['stroke'].map({0:'no', 1:'yes'})

# Quick check
print(df.isna().sum())


age                  0
hypertension         0
heart_disease        0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64


### Discretize Continuous Columns
We convert continuous values into categories (bins).
Bayesian Networks work better with discrete features.

In [16]:
disc_cols = ['age','avg_glucose_level','bmi']
disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df[disc_cols] = disc.fit_transform(df[disc_cols]).astype(int).astype(str)

# Make sure all columns are strings (categorical)
for c in df.columns:
    df[c] = df[c].astype(str)

In [17]:
from sklearn.utils import resample

stroke_yes = df[df['stroke'] == 'yes']
stroke_no = df[df['stroke'] == 'no']

stroke_yes_up = resample(stroke_yes, 
                         replace=True,
                         n_samples=len(stroke_no),
                         random_state=42)

df_balanced = pd.concat([stroke_yes_up, stroke_no]).sample(frac=1, random_state=42)
df = df_balanced.reset_index(drop=True)

### Split Data
We split data into training and testing parts.  
Train data builds the model and test data checks accuracy.


In [18]:
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['stroke'])
print("Train/test sizes:", train_df.shape, test_df.shape)

# List of variables
vars_all = list(train_df.columns)
print("Variables used:", vars_all)

Train/test sizes: (7291, 7) (2431, 7)
Variables used: ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']


### Handle Class Imbalance
We oversample stroke cases to balance the dataset.  
This helps the model learn stroke patterns properly.


### Helper Functions
These functions support structure learning and scoring.  
Used inside the learning algorithm.


In [19]:
def chi2_indep(x, y):
    table = pd.crosstab(x, y)
    if table.size == 0 or table.shape[0] < 2 or table.shape[1] < 2:
        return 1.0
    chi2, p, _, _ = chi2_contingency(table)
    return p

def ci_test(df, X, Y, Z=None, alpha=0.05):
    if not Z:
        return chi2_indep(df[X], df[Y]) >= alpha
    total_chi2 = 0.0
    total_dof = 0
    for _, sub in df.groupby(Z):
        if sub.shape[0] < 5:
            continue
        table = pd.crosstab(sub[X], sub[Y])
        if table.size == 0 or table.shape[0] < 2 or table.shape[1] < 2:
            continue
        chi2, p, dof, _ = chi2_contingency(table)
        total_chi2 += chi2
        total_dof += dof
    if total_dof == 0:
        return False
    p_comb = 1 - math.exp(math.log(1 - 0.0) * 1)  # placeholder - rather compute via chi2 cdf
    # more straightforward:
    from scipy.stats import chi2 as chi2dist
    p_comb = 1 - chi2dist.cdf(total_chi2, total_dof)
    return p_comb >= alpha

def has_cycle(edges, nodes):
    adj = {n:[] for n in nodes}
    for u,v in edges:
        adj[u].append(v)
    visited = {n:0 for n in nodes}
    def dfs(u):
        if visited[u]==1: return True
        if visited[u]==2: return False
        visited[u]=1
        for w in adj[u]:
            if dfs(w): return True
        visited[u]=2
        return False
    for n in nodes:
        if visited[n]==0 and dfs(n): return True
    return False

def compute_cpt_counts(df, child, parents):
    res = {}
    if not parents:
        c = df[child].value_counts()
        res[()] = c.to_dict()
    else:
        for pvals, group in df.groupby(parents):
            key = pvals if isinstance(pvals, tuple) else (pvals,)
            res[key] = group[child].value_counts().to_dict()
    return res

def bic_score(df, edges, nodes):
    N = len(df)
    parents = {n:[] for n in nodes}
    for u,v in edges:
        parents[v].append(u)
    LL = 0.0
    num_params = 0
    for child in nodes:
        child_states = sorted(df[child].unique())
        par = parents[child]
        if not par:
            counts = df[child].value_counts().to_dict()
            total = sum(counts.values())
            for s in child_states:
                c = counts.get(s, 0)
                if c>0:
                    p = c/total
                    LL += c * math.log(p)
            num_params += (len(child_states)-1)
        else:
            grouped = df.groupby(par)
            for _, subset in grouped:
                total = len(subset)
                if total == 0: continue
                counts = subset[child].value_counts().to_dict()
                for s in child_states:
                    c = counts.get(s, 0)
                    if c>0:
                        p = c/total
                        LL += c * math.log(p)
                num_params += (len(child_states)-1)
    bic = LL - 0.5 * num_params * math.log(N)
    return bic


In [20]:
init_edges = {
    ('age','stroke'),
    ('hypertension','stroke'),
    ('heart_disease','stroke'),
    ('avg_glucose_level','stroke')
}


### Learn BN Structure
We use hill-climbing to find the best network shape.  
The model improves step by step using BIC score.


In [21]:
def hill_climb_structure(df, nodes, init_edges=set(), max_iter=200):
    edges = set(init_edges)
    best = bic_score(df, edges, nodes)
    it = 0
    while it < max_iter:
        it += 1
        improved = False
        best_move = None
        best_score = best
        # consider add
        for u,v in permutations(nodes,2):
            if (u,v) in edges: continue
            cand = set(edges); cand.add((u,v))
            if has_cycle(cand, nodes): continue
            score = bic_score(df, cand, nodes)
            if score > best_score:
                best_score = score; best_move = ('add',(u,v))
        # consider remove
        for (u,v) in list(edges):
            cand = set(edges); cand.remove((u,v))
            score = bic_score(df, cand, nodes)
            if score > best_score:
                best_score = score; best_move = ('remove',(u,v))
        # reverse
        for (u,v) in list(edges):
            cand = set(edges); cand.remove((u,v)); cand.add((v,u))
            if has_cycle(cand, nodes): continue
            score = bic_score(df, cand, nodes)
            if score > best_score:
                best_score = score; best_move = ('reverse',(u,v))
        if best_move:
            typ,edge = best_move
            if typ=='add': edges.add(edge)
            elif typ=='remove': edges.remove(edge)
            else:
                u,v = edge; edges.remove((u,v)); edges.add((v,u))
            best = best_score
            improved = True
        if not improved:
            break
    return edges, best

In [22]:
edges, score = hill_climb_structure(sample, nodes, init_edges=init_edges, max_iter=80)

NameError: name 'sample' is not defined

### Learn Probabilities
We calculate CPTs using training data.  
These probabilities help in predicting stroke risk.


In [ ]:
def mle_learn_cpts(df, edges, nodes, laplace=1e-6):
    parents = {n:[] for n in nodes}
    for u,v in edges:
        parents[v].append(u)
    cpts = {}
    for child in nodes:
        par = parents[child]
        counts = compute_cpt_counts(df, child, par)
        child_vals = sorted(df[child].unique())
        cpt = {}
        for pc in counts:
            total = sum(counts[pc].values())
            probs = {}
            for val in child_vals:
                probs[val] = (counts[pc].get(val,0)+laplace) / (total + laplace*len(child_vals))
            cpt[pc] = probs
        cpts[child] = {'parents': par, 'cpt': cpt}
    return cpts

In [23]:
def predict_prob_from_cpts(cpts, var_domains, evidence, target='stroke'):
    # returns probability P(target='yes' | evidence)
    # build full enumeration over parents of target if necessary
    info = cpts[target]
    parents = info['parents']
    child_vals = var_domains[target]
    target_yes = 'yes'
    # find parent values we have (if any parent not in evidence, enumerate)
    missing_parents = [p for p in parents if p not in evidence]
    probs_sum = 0.0
    total_weight = 0.0
    # enumerate over all possible assignments to missing parents
    if missing_parents:
        vals_list = [var_domains[p] for p in missing_parents]
        for combo in product(*vals_list):
            # build parent tuple in same order as stored
            full_par_vals = []
            i=0
            for p in parents:
                if p in evidence:
                    full_par_vals.append(evidence[p])
                else:
                    full_par_vals.append(combo[i]); i+=1
            key = tuple(full_par_vals) if len(full_par_vals)>1 else (full_par_vals[0],)
            probs = info['cpt'].get(key, None)
            if not probs:
                # fallback to uniform
                prob_yes = 1.0/len(child_vals)
            else:
                prob_yes = probs.get(target_yes, 0.0)
            probs_sum += prob_yes
            total_weight += 1.0
        return probs_sum / total_weight if total_weight>0 else 0.0
    else:
        key = tuple(evidence[p] for p in parents) if parents else ()
        probs = info['cpt'].get(key, None)
        if not probs:
            return 0.0
        return probs.get(target_yes, 0.0)

### Train Ensemble Models
We train multiple BNs using bootstrap samples.  
This reduces errors and improves stability.


In [24]:
def train_ensemble(train_df, nodes, M=7):
    ensemble = []
    bic_scores = []
    start = time.time()
    for m in range(M):
        sample = train_df.sample(frac=1.0, replace=True, random_state=42+m)
        # hill-climb structure (start empty), limit iterations for speed
        edges, score = hill_climb_structure(sample, nodes, max_iter=80)
        cpts = mle_learn_cpts(sample, edges, nodes)
        ensemble.append({'edges': edges, 'cpts': cpts, 'bic': score})
        bic_scores.append(score)
        print(f"Trained model {m+1}/{M}  (BIC {score:.2f})")
    print("Ensemble training time:", time.time()-start)
    return ensemble

# choose small nodes to speed up (use the columns we have)
nodes = vars_all  # e.g., ['age','hypertension','heart_disease','avg_glucose_level','bmi','smoking_status','stroke']

# For speed, you may remove less informative nodes; but we'll try with these 7
ensemble = train_ensemble(train_df, nodes, M=5)

Trained model 1/5  (BIC -37442.74)
Trained model 2/5  (BIC -37221.41)
Trained model 3/5  (BIC -37178.23)
Trained model 4/5  (BIC -37171.08)
Trained model 5/5  (BIC -37102.07)
Ensemble training time: 200.11134815216064


### Predict Stroke
We combine predictions from all BN models.  
This gives better results than a single model.


In [25]:
var_domains = {c: sorted(train_df[c].unique()) for c in train_df.columns}

# ===========================
# 11. Predict on test set: average and BIC-weighted average
# ===========================
y_true = []
y_pred_avg = []
y_pred_bma = []
prob_list_avg = []
prob_list_bma = []

# compute weights from BIC (convert to positive weights; higher bic -> higher weight)
bics = np.array([m['bic'] for m in ensemble])
# to avoid numerical issues, shift
w = np.exp(bics - bics.max())
weights = w / w.sum()

for idx,row in test_df.iterrows():
    ev = {c: row[c] for c in row.index if c!='stroke'}
    y_true.append(row['stroke'])
    probs = []
    for model in ensemble:
        prob = predict_prob_from_cpts(model['cpts'], var_domains, ev, target='stroke')
        probs.append(prob)
    # average
    p_avg = np.mean(probs)
    p_bma = np.dot(weights, probs)
    prob_list_avg.append(p_avg)
    prob_list_bma.append(p_bma)
    y_pred_avg.append('yes' if p_avg>0.5 else 'no')
    y_pred_bma.append('yes' if p_bma>0.5 else 'no')

### Evaluate Performance
We measure how well the model predicts stroke.  
Accuracy, precision, recall, and AUC are reported.


In [26]:
def evaluate(y_true, y_pred, probs=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label='yes')
    rec = recall_score(y_true, y_pred, pos_label='yes')
    f1 = f1_score(y_true, y_pred, pos_label='yes')
    auc = roc_auc_score([1 if y=='yes' else 0 for y in y_true], probs) if probs is not None else None
    print("Accuracy:", acc)
    print("Precision:", prec)
    print("Recall:", rec)
    print("F1:", f1)
    if auc is not None: print("AUC:", auc)
    print("Confusion matrix:\n", confusion_matrix(y_true, y_pred, labels=['no','yes']))

print("==== Simple Average Ensemble ====")
evaluate(y_true, y_pred_avg, prob_list_avg)
print("\n==== BIC-weighted Ensemble (approx BMA) ====")
evaluate(y_true, y_pred_bma, prob_list_bma)

==== Simple Average Ensemble ====
Accuracy: 0.7786918963389552
Precision: 0.7352328005559416
Recall: 0.8707818930041152
F1: 0.7972871137905049
AUC: 0.8445222817847087
Confusion matrix:
 [[ 835  381]
 [ 157 1058]]

==== BIC-weighted Ensemble (approx BMA) ====
Accuracy: 0.7803373097490744
Precision: 0.7336993822923816
Recall: 0.8798353909465021
F1: 0.8001497005988024
AUC: 0.8222844920944337
Confusion matrix:
 [[ 828  388]
 [ 146 1069]]
